In [1]:
import re
import math
import os
import hashlib
import random
import numpy as np
import tensorflow as tf
from tensorflow.python.platform import gfile
from tensorflow.python.ops import io_ops
from tensorflow.contrib.framework.python.ops import audio_ops as contrib_audio
from tensorflow.python.util import compat

sess = tf.InteractiveSession()



In [7]:
tom_words = ["hey","there"]
wanted_words = ["hey"]

In [16]:
mfcc = tf.placeholder(tf.float32,[None,98,40])
labels_ph = tf.placeholder(tf.float32,[None,len(tom_words)],name="labels_ph")
train_indices_ph = tf.placeholder(tf.int32,[None]) # used for train accuracy
val_indices_ph = tf.placeholder(tf.int32,[None]) # used for val accuracy

keep_prob = tf.placeholder(tf.float32,name="keep_prob") # will be 0.5 for training, 1 for test

fingerprint_4d = tf.reshape(mfcc,[-1,mfcc.shape[1],mfcc.shape[2],1])

conv_1_channels = 64
conv_2_channels = 64

weights_1 = tf.Variable(tf.truncated_normal([20,8,1,conv_1_channels],stddev=0.01)) # [height,width,depth,channels]
bias_1 = tf.Variable(tf.zeros([64]))
conv_1 = tf.nn.conv2d(fingerprint_4d,weights_1,[1,1,1,1],"SAME") + bias_1
relu_1 = tf.nn.relu(conv_1)
dropout_1 = tf.nn.dropout(relu_1,keep_prob)
maxpool_1 = tf.nn.max_pool(dropout_1,[1,2,2,1],[1,2,2,1],"SAME")

weights_2 = tf.Variable(tf.truncated_normal([10,4,conv_1_channels,conv_2_channels],stddev=0.01)) # [height,width,first_conv_channels,second_conv_channels]
bias_2 = tf.Variable(tf.zeros([64]))
conv_2 = tf.nn.conv2d(maxpool_1,weights_2,[1,1,1,1],"SAME") + bias_2
relu_2 = tf.nn.relu(conv_2)
dropout_2 = tf.nn.dropout(relu_2,keep_prob)

_ , now_height, now_width, _ = dropout_2.get_shape()
now_height = int(now_height)
now_width = int(now_width)
now_flat_elements = now_height * now_width * conv_2_channels

flat_layer = tf.reshape(dropout_2,[-1,now_flat_elements])


weights_3 = tf.Variable(tf.truncated_normal([now_flat_elements,len(tom_words)],stddev=0.01))
bias_3 = tf.Variable(tf.zeros([len(tom_words)]))
final_layer = tf.matmul(flat_layer,weights_3) + bias_3

# using sigmoid here!! (bc classes are no longer mutually exclusive)
cross_entropy_loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=labels_ph, logits=final_layer)
cross_entropy_mean = tf.reduce_mean(cross_entropy_loss)

learning_rate_ph = tf.placeholder(tf.float32,[],name="learning_rate_ph")
train_step = tf.train.GradientDescentOptimizer(learning_rate_ph).minimize(cross_entropy_mean)

unknown_val = final_layer[:,1] # bc unknown is the second label for all unknown words
train_predicted_indices = tf.argmax(final_layer,axis=1,output_type=tf.int32)
val_predicted_indices = tf.where(train_predicted_indices >= len(wanted_words),1,train_predicted_indices)

train_correct_prediction = tf.equal(train_predicted_indices,train_indices_ph)
val_correct_prediction = tf.equal(val_predicted_indices,val_indices_ph)

# confusion_matrix = tf.confusion_matrix(val_indices,predicted_indices,num_classes=label_elements)

# don't take unknown into account at all for this
train_accuracy = tf.reduce_mean(tf.cast(train_correct_prediction,tf.float32))
# take unknown into account, and don't take any unknown words into account
val_accuracy = tf.reduce_mean(tf.cast(val_correct_prediction,tf.float32))

global_step = tf.train.get_or_create_global_step()
increment_global_step = tf.assign(global_step,global_step + 1)

saver = tf.train.Saver(tf.global_variables())

tf.summary.scalar("cross_entropy",cross_entropy_mean)
tf.summary.scalar("train_accuracy",train_accuracy)
tf.summary.scalar("val_accuracy",val_accuracy)
merged_summaries = tf.summary.merge_all()
train_writer = tf.summary.FileWriter("logs/train",sess.graph)
val_writer = tf.summary.FileWriter("logs/val",sess.graph)

tf.logging.set_verbosity(tf.logging.INFO)
sess.run(tf.global_variables_initializer())




ValueError: Shapes must be equal rank, but are 0 and 1 for 'Select_3' (op: 'Select') with input shapes: [?], [], [?].

In [13]:
val_predicted_indices.shape

NameError: name 'val_predicted_indices' is not defined

In [14]:
train_predicted_indices.dtype

tf.int64

In [17]:
tf.tile(1,10)

ValueError: Shape must be rank 1 but is rank 0 for 'Tile' (op: 'Tile') with input shapes: [], [].